<a href="https://colab.research.google.com/github/MohammadrezaPourreza/Seq2Seq/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we should download data from github and install some package

In [ ]:
!git clone https://github.com/HamidSajjadi/Pubmed-T5Gan.git
!pip install transformers
!pip install sent2vec
!pip install biobert_embedding
!pip install torchtext

In this part we import requried packages for our project

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.legacy.datasets import Multi30k
import torchtext.vocab as vocab
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
import numpy as np
import random
import string
import math
import time
import pandas as pd
import pickle
import sent2vec
from random import random as rand
from tqdm import tqdm
from torch.utils.data import DataLoader
from biobert_embedding.embedding import BiobertEmbedding
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
from gensim.models import FastText as ft
from gensim.test.utils import datapath

Setting seeds

In [3]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Then we should load data and process it to be suitable for Encoder

In [4]:
#open dataset with pandas and drop unrelated columns, removing punctuations from keywords
df = pd.read_csv('/content/Pubmed-T5Gan/data/clef.csv')
df = df.sample(frac=1)
df = df.drop(['topic_id', 'related','orig'], axis=1)
df['keywords'] = df['keywords'].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)))
df['description'] = df['description'].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)))
test_df,val_df,train_df = np.split(df, [int(.1*len(df)), int(.2*len(df))])
test_df.to_csv('test.csv', encoding='utf-8',index=False,header=False)
val_df.to_csv('val.csv', encoding='utf-8',index=False,header=False)
train_df.to_csv('train.csv', encoding='utf-8',index=False,header=False)


## **Data Augmentation**

**Install nlpaug package**

In [ ]:
!pip install numpy git+https://github.com/makcedward/nlpaug.git
!pip install nltk>=3.4.5
import nltk
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install pandas
import pandas as pd

**Word augmentation using pre trained models**

In [ ]:
train_df = pd.read_csv('train.csv',header=None)
aug = naw.ContextualWordEmbsAug(
    model_path='bert-base-uncased', action="substitute")
augmented = []
for index, row in train_df.iterrows():
    org = row[0]
    augmented_text = aug.augment(org,n=5)
    temp = []
    temp.append(org)
    temp.append(row[1])
    augmented.append(temp)
    for sent in augmented_text:
      temp = []
      temp.append(sent)
      temp.append(row[1])
      augmented.append(temp)
trainAug_df = pd.DataFrame(augmented)
trainAug_df.to_csv('trainAug.csv', encoding='utf-8',index=False,header=False)

**Creating bioBert word embedding matix**

In [ ]:
biobert = BiobertEmbedding()
Bertvocab = open("/content/biobert_v1.1_pubmed_pytorch_model/vocab.txt", "r")
vocab = Bertvocab.read()
with open('biobert_embeddings.txt', 'w+') as f:
  for word in vocab.splitlines():
    try:
      word_vec = biobert.word_vector(str(word))[0].flatten().tolist()
      vector_str = ' '.join([str(v) for v in word_vec])
      f.write(f'{word} {vector_str}\n')
    except:
      print(word)
      #print(biobert.word_vector(str(word)))


**Or download it from my drive and share your drive here**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Prepocessing the data for encoder and tokenizing it


In [ ]:
#download spacy models
spacy_en = spacy.load('en_core_web_sm')
#creating tokenizing function
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
Text = Field(tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True)
train_data, valid_data, test_data = TabularDataset.splits(path = './',
    train='trainAug.csv',
    validation='val.csv', test='test.csv', format='csv',
    fields=[('description', Text), ('keywords', Text)])
#use pretrainded Glove embeddings
#Text.build_vocab(train_data,vectors="glove.6B.50d")
#use pretrainded biobert embeddings
biobert_embeddings = vocab.Vectors(name = 'drive/MyDrive/biobert_embeddings.txt',cache='./content/')
Text.build_vocab(train_data, vectors = biobert_embeddings)
print(f"Unique tokens in target (keywrods) vocabulary: {len(Text.vocab)}")

#TPU
#device = xm.xla_device()

#GPU and CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key = lambda x: len(x.description),
    sort_within_batch=True)

**checking the embedding**

In [ ]:
vocab_len,emb_dim = Text.vocab.vectors.size()
print(emb_dim)
print(vocab_len)
biobert = BiobertEmbedding()
print(torch.equal(biobert.word_vector(str('the'))[0],Text.vocab.vectors[Text.vocab.stoi["the"]]))
#2 is thenumber of special tokens
unk_num = 0
for i in range (2,vocab_len):
  if len(Text.vocab.vectors[i, :].nonzero()) == 0:
    unk_num += 1
print("number of unknown words = " + str(unk_num))

**Initializing unknown words randomly**

In [ ]:
def init_emb(vocab, init="randn", num_special_toks=2):
    emb_vectors = vocab.vectors
    sweep_range = len(vocab)
    running_norm = 0.
    num_non_zero = 0
    total_words = 0
    for i in range(num_special_toks, sweep_range):
        if len(emb_vectors[i, :].nonzero()) == 0:
          if init == "randn":
              torch.nn.init.normal(emb_vectors[i], mean=0, std=0.05)
        else:
          num_non_zero += 1
          running_norm += torch.norm(emb_vectors[i])
        total_words += 1
    print("average norm is {}, number of known words are {}, total number of words are {}".format(
        running_norm / num_non_zero, num_non_zero, total_words))
init_emb(Text.vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  # Remove the CWD from sys.path while we load stuff.


average norm is 73.70836639404297, number of known words are 3685, total number of words are 12386


**Building a sequence to sequence model**

**Encoder part without attention**

In [ ]:
class Encoder(nn.Module):
    def __init__(self,field , hid_dim, n_layers, dropout):
        super().__init__()
        self.field = field
        self.input_dim,self.emb_dim = self.field.vocab.vectors.size()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        #for trainable embedding layer uncomment below
        #self.embedding = nn.Embedding.from_pretrained(field.vocab.vectors, freeze=False)
        #pretrained embedding
        self.embedding = nn.Embedding.from_pretrained(self.field.vocab.vectors, freeze=False)
        #self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        #LSTM
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers, dropout = dropout,bidirectional = True)
        #GRU
        #self.rnn = nn.GRU(self.emb_dim, self.hid_dim, self.n_layers, dropout = dropout,bidirectional = False)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        #src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        #LSTM
        outputs, (hidden, cell) = self.rnn(embedded)
        #GRU
        #outputs , hidden = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer
        #LSTM
        return hidden, cell
        #GRU
        #return hidden

**Decoder without attention**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, field, hid_dim, n_layers, dropout):
        super().__init__()
        self.field = field
        self.output_dim, self.emb_dim = self.field.vocab.vectors.size()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding.from_pretrained(self.field.vocab.vectors, freeze=False)
        #LSTM
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers, dropout = dropout,bidirectional = True)
        #GRU
        #self.rnn = nn.GRU(self.emb_dim, self.hid_dim, self.n_layers, dropout = dropout,bidirectional = False)
        self.fc_out = nn.Linear(self.hid_dim*2, self.output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, input, hidden, cell = None):
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]     
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        input = input.unsqueeze(0)
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
        #LSTM
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        #GRU
        #output,hidden = self.rnn(embedded,hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        #LSTM
        return prediction, hidden, cell
        #GRU
        #return prediction, hidden

**Seq2Seq class without attention**

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        #LSTM
        hidden, cell = self.encoder(src)
        #GRU
        #hidden = self.encoder(src)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        for t in range(1, trg_len):
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            #LSTM
            output, hidden, cell = self.decoder(input, hidden, cell)
            #GRU
            #output, hidden = self.decoder(input,hidden)
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        return outputs

**Creating Encoder, decoder, attention and seq2seq all together**

In [ ]:
#encoder
class Encoder(nn.Module):
    def __init__(self, field, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()        
        self.field = field
        self.input_dim,self.emb_dim = self.field.vocab.vectors.size() 
        self.embedding = nn.Embedding.from_pretrained(self.field.vocab.vectors, freeze=False)   
        self.rnn = nn.GRU(self.emb_dim, enc_hid_dim, bidirectional = True)        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)       
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):        
        #src = [src len, batch size]       
        embedded = self.dropout(self.embedding(src))       
        #embedded = [src len, batch size, emb dim]       
        outputs, hidden = self.rnn(embedded)               
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        return outputs, hidden
#attention
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)
#decoder
class Decoder(nn.Module):
    def __init__(self, field, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.attention = attention
        self.field = field
        self.output_dim,self.emb_dim = self.field.vocab.vectors.size() 
        self.embedding = nn.Embedding.from_pretrained(self.field.vocab.vectors, freeze=False)        
        self.rnn = nn.GRU((enc_hid_dim * 2) + self.emb_dim, dec_hid_dim)     
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, self.output_dim)  
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]        
        input = input.unsqueeze(0)       
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)
#seq2seq
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs


# **Training the Seq2Seq model**

**initialization without attention**

In [ ]:
HID_DIM = 64
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(Text, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(Text, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

**initilization with attention**

In [ ]:
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(Text, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(Text, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

**Initialize the wieghts**

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(12388, 768)
    (rnn): GRU(768, 64, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=192, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
    (embedding): Embedding(12388, 768)
    (rnn): GRU(896, 64)
    (fc_out): Linear(in_features=960, out_features=12388, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

**trainable features**

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 31,458,468 trainable parameters


**Focal loss for imbalanced dataset**

In [ ]:
class FocalLoss(nn.modules.NLLLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean',ignore_index = -100):
        super(FocalLoss, self).__init__(weight,reduction=reduction,ignore_index= ignore_index)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

**Optimizer and Loss function**

In [ ]:
#adding regularization
optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
#without regularization
#optimizer = optim.Adam(model.parameters())
Text_PAD_IDX = Text.vocab.stoi[Text.pad_token]
#focal loss
#criterion = FocalLoss(ignore_index = Text_PAD_IDX)
#crossentropy
criterion = nn.CrossEntropyLoss(ignore_index = Text_PAD_IDX)

**trainnig method**

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.description
        trg = batch.keywords
        optimizer.zero_grad()
        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

**Evaluation method**

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.description
            trg = batch.keywords
            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

**Epoch time**

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

**Training the model**

In [ ]:
N_EPOCHS = 40
CLIP = 1
#model.load_state_dict(torch.load('tut1-model.pt'))
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


**Test the model**

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt', map_location='cpu'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

**Create output method**

In [ ]:
def create_query(sentence, src_field, trg_field, model, device, max_len = 150):

    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

**test the code**

In [ ]:
for i in range (10,20):
  src = vars(test_data.examples[i])['description']
  trg = vars(test_data.examples[i])['keywords']
  print(f'src = {src}')
  print(f'trg = {trg}')
  query = create_query(src, Text, Text, model, device)
  print(f'predicted trg = {query}')

Refrences : 
https://github.com/bentrevett/pytorch-seq2seq